<a href="https://colab.research.google.com/github/yanesx96/coursera_capstone/blob/main/agrupamiento_y_segmentacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Jupyter Notebook para asignacion de agrupacion y segmentacion de vencindarios de toronto usando web scraping

In [2]:
import pandas as pd
import numpy as np
import requests

from bs4 import BeautifulSoup

In [3]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source, 'html5lib')

postal_codes_dict = {} # diccionario que contendra los codigos postales

In [4]:
for table_cell in soup.find_all('td'):
    try:
        postal_code = table_cell.p.b.text # se obtiene el codigo postal
        postal_code_investigate = table_cell.span.text
        neighborhoods_data = table_cell.span.text # se obtiene la informacion de la celda
        borough = neighborhoods_data.split('(')[0] # se obtiene el distrito
        
        # si la celda aparece como no asignada se ignora 
        if neighborhoods_data == 'Not assigned':
            neighborhoods = []
        # si aparece asignada se agrega al diccionario
        else:
            postal_codes_dict[postal_code] = {}
            
            try:
                neighborhoods = neighborhoods_data.split('(')[1]
            
                # se remueven los parentesis
                neighborhoods = neighborhoods.replace('(', ' ')
                neighborhoods = neighborhoods.replace(')', ' ')

                neighborhoods_names = neighborhoods.split('/')
                neighborhoods_clean = ', '.join([name.strip() for name in neighborhoods_names])
            except:
                borough = borough.strip('\n')
                neighborhoods_clean = borough
 
            # se agrega el distrito al diccionario
            postal_codes_dict[postal_code]['borough'] = borough
            postal_codes_dict[postal_code]['neighborhoods'] = neighborhoods_clean
    except:
        pass

In [5]:
# se crea el dataframe
columns = ['PostalCode', 'Borough', 'Neighborhood']
toronto_data = pd.DataFrame(columns=columns)
toronto_data

,PostalCode,Borough,Neighborhood


In [6]:
# se llena el marco de datos con el diccionario
for ind, postal_code in enumerate(postal_codes_dict):
    borough = postal_codes_dict[postal_code]['borough']
    neighborhood = postal_codes_dict[postal_code]['neighborhoods']
    toronto_data = toronto_data.append({"PostalCode": postal_code, 
                                        "Borough": borough, 
                                        "Neighborhood": neighborhood},
                                        ignore_index=True)

In [7]:
# se imprime el numero de filas del dataframe
toronto_data.shape[0]
toronto_data

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."
